## Instalando las dependencias necesarias

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

#### Definiendo variables de entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=97240167b94f890e12df2fe24ca94e2acda63308880298a4a2af3e1d5f037aba
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

## Trabajar con datos

In [ ]:
# Para obtener detalle sobre un función de pyspark
help(spark.read.text)

Help on method text in module pyspark.sql.readwriter:

text(paths: Union[str, List[str]], wholetext: bool = False, lineSep: Optional[str] = None, pathGlobFilter: Union[bool, str, NoneType] = None, recursiveFileLookup: Union[bool, str, NoneType] = None, modifiedBefore: Union[bool, str, NoneType] = None, modifiedAfter: Union[bool, str, NoneType] = None) -> 'DataFrame' method of pyspark.sql.readwriter.DataFrameReader instance
    Loads text files and returns a :class:`DataFrame` whose schema starts with a
    string column named "value", and followed by partitioned columns if there
    are any.
    The text files must be encoded as UTF-8.
    
    By default, each line in the text file is a new row in the resulting DataFrame.
    
    .. versionadded:: 1.6.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    paths : str or list
        string, or list of strings, for input path(s).
    
    Other Parameters
    ----------------
    Ex

In [ ]:
# Ver el archivo por pantalla y no truncarlo
spark.read.text('/content/drive/MyDrive/retail_db/schemas.json', wholetext=True).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Ver el primer registro del archivo por pantalla , ademas traerlo en una variable de tipo texto
schema_text = spark.read.text('/content/drive/MyDrive/retail_db/schemas.json', wholetext=True).first().value

schema_text

'{\r\n    "departments": [\r\n        {\r\n            "column_name": "department_id",\r\n            "data_type": "integer",\r\n            "column_position": 1\r\n        },\r\n        {\r\n            "column_name": "department_name",\r\n            "data_type": "string",\r\n            "column_position": 2\r\n        }\r\n    ],\r\n    "categories": [\r\n        {\r\n            "column_name": "category_id",\r\n            "data_type": "integer",\r\n            "column_position": 1\r\n        },\r\n        {\r\n            "column_name": "category_department_id",\r\n            "data_type": "integer",\r\n            "column_position": 2\r\n        },\r\n        {\r\n            "column_name": "category_name",\r\n            "data_type": "string",\r\n            "column_position": 3\r\n        }\r\n    ],\r\n    "orders": [\r\n        {\r\n            "column_name": "order_id",\r\n            "data_type": "integer",\r\n            "column_position": 1\r\n        },\r\n        {\r\n 

In [ ]:
type(schema_text)

str

In [ ]:
# Libreria de python usada para manejar los archivos json
import json

In [ ]:
#  Lectura del json como un python dict
column_details = json.loads(schema_text)['orders']
column_details

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [ ]:
# Se ordena los datos por la columns position
sorted(column_details, key=lambda col: col['column_position'])

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [ ]:
# Extraer del diccionario del nombre de la columna
columns = [info['column_name'] for info in sorted(column_details, key=lambda col: col['column_position'])]

In [ ]:
help(spark.read.csv)

Help on method csv in module pyspark.sql.readwriter:

csv(path: Union[str, List[str]], schema: Union[pyspark.sql.types.StructType, str, NoneType] = None, sep: Optional[str] = None, encoding: Optional[str] = None, quote: Optional[str] = None, escape: Optional[str] = None, comment: Optional[str] = None, header: Union[bool, str, NoneType] = None, inferSchema: Union[bool, str, NoneType] = None, ignoreLeadingWhiteSpace: Union[bool, str, NoneType] = None, ignoreTrailingWhiteSpace: Union[bool, str, NoneType] = None, nullValue: Optional[str] = None, nanValue: Optional[str] = None, positiveInf: Optional[str] = None, negativeInf: Optional[str] = None, dateFormat: Optional[str] = None, timestampFormat: Optional[str] = None, maxColumns: Union[str, int, NoneType] = None, maxCharsPerColumn: Union[str, int, NoneType] = None, maxMalformedLogPerPartition: Union[str, int, NoneType] = None, mode: Optional[str] = None, columnNameOfCorruptRecord: Optional[str] = None, multiLine: Union[bool, str, NoneType] 

In [ ]:
# Lectura csv, permitir que infiera el esquema de los datos -  asigna los nombres anteriores a las columns
orders = spark.read.csv('/content/drive/MyDrive/retail_db/orders', inferSchema=True).toDF(*columns)

In [ ]:
from pyspark.sql.functions import count, col

In [ ]:
# Aplicación de funciones de agregación en pyspark
orders.\
  groupBy('order_status').\
    agg(count('*').alias('order_count')).\
      orderBy(col('order_count').desc()).\
      show()

+---------------+-----------+
|   order_status|order_count|
+---------------+-----------+
|       COMPLETE|      22899|
|PENDING_PAYMENT|      15030|
|     PROCESSING|       8275|
|        PENDING|       7610|
|         CLOSED|       7556|
|        ON_HOLD|       3798|
|SUSPECTED_FRAUD|       1558|
|       CANCELED|       1428|
| PAYMENT_REVIEW|        729|
+---------------+-----------+

